# Uniswap V3 price analysis

In this notebook we will show how to download price events from Uniswap V3 to your computer as CSV files and use them to analyse price in each pool.

You will need to havev Ethereum JSON-RPC API node URL in order to pull out the data from Ethereum blockchain.
You will be interactively asked for this API key.

### Setup the notebook

In [300]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [301]:
import pandas as pd
import matplotlib.pyplot as plt

from eth_defi.uniswap_v3.constants import UNISWAP_V3_FACTORY_CREATED_AT_BLOCK
from eth_defi.uniswap_v3.events import fetch_events_to_csv

### Download the raw data

For simplicity, you can sign up for free access to an Ethereum node for example at [Infura](https://infura.io/), however we recommend you to [run your own Ethereum node](https://tradingstrategy.ai/blog/preparing-a-server-for-hosting-goethereum-full-node).

In [302]:
from web3 import Web3, HTTPProvider

# Get your node JSON-RPC URL
json_rpc_url = input("Please enter your Ethereum mainnet JSON-RPC URL here")
web3 = Web3(HTTPProvider(json_rpc_url))

As an example, here we download raw events from first few blocks after Uniswap V3 was deployed.
The events will be stored in several CSV files at `/tmp` folder.

The scan will take hours. It can resumed on a crash, as we save the last scanned block in a JSON state file.

In [303]:
from eth_defi.event_reader.json_state import JSONFileScanState

start_block = UNISWAP_V3_FACTORY_CREATED_AT_BLOCK
end_block = UNISWAP_V3_FACTORY_CREATED_AT_BLOCK + 250_000

# Stores the last block number of event data we store
state = JSONFileScanState("/tmp/uniswap-v3-price-scan.json")

fetch_events_to_csv(json_rpc_url, state, start_block=start_block, end_block=end_block)

Restored previous scan state, data until block 12,619,621, we are skipping 250,000 blocks out of 250,000 total
Scanning block range 12,619,621 - 12,619,621


0it [00:00, ?it/s]

Wrote 0 PoolCreated events
Wrote 0 Swap events
Wrote 0 Mint events
Wrote 0 Burn events


### Analysing Uniswap v3 price formation

In Uniswap V3, you can get the current price of any pool from any given moment using swap events.

In [304]:
swap_df = pd.read_csv("/tmp/uniswap-v3-swap.csv")

print(f"We have total {len(swap_df):,} swaps in the dataset")

We have total 1,229,369 swaps in the dataset


Choose a pool to analyse, for example: USDC/ETH 0.3%.
You can find pool addresses [on Uniswap v3 info site](https://info.uniswap.org/#/).

In [305]:
from eth_defi.uniswap_v3.pool import fetch_pool_details

pool_address = "0x8ad599c3A0ff1De082011EFDDc58f1908eb6e6D8"
pool_details = fetch_pool_details(web3, pool_address)

print(pool_details)

Pool 0x8ad599c3A0ff1De082011EFDDc58f1908eb6e6D8 is USDC-WETH, with the fee 0.3000%


Extract the swaps of this pool from the Uniswap v3 full dataset of all pools.

In [306]:
df = swap_df.loc[swap_df.pool_contract_address == pool_address.lower()]
df

,block_number,timestamp,tx_hash,log_index,pool_contract_address,amount0,amount1,sqrt_price_x96,liquidity,tick
19,12371376,2021-05-05T01:56:23,0xce7c3c307d820785caa12938012372fc9366a614a6aa...,26,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,1157920892373161954235709850086879078532699846...,100000000000000,1377932816571815120446551350158799,4303369674465501,195285
21,12373132,2021-05-05T08:23:26,0x9a1c51b0bffbf840948f3b6e3f3e495ba1cd3fa64854...,192,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,1157920892373161954235709850086879078532699846...,50000000000000000,1378850591292581266780357299649652,4303369674465501,195298
25,12373520,2021-05-05T09:50:51,0xc58715c62a5bf70a6ca09f0e51546d6cad76c8d4fff0...,8,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,1157920892373161954235709850086879078532699846...,100000000000000,1378852426842022799073024911548633,4303369674465501,195298
26,12374077,2021-05-05T11:59:57,0x288c21b8b4fbf449b1d086a06e43b124ac2bc088c3f5...,86,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,2,1157920892373161954235709850086879078532699846...,1378852426842016741051966412054516,4304946248093346,195298
45,12375770,2021-05-05T18:09:23,0xf76f19b991131f0f396f116f79215637dacad3cf5e1b...,249,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,1157920892373161954235709850086879078532699846...,300000000000000,1370242002332554394456531434422627,52976694435070763,195173
...,...,...,...,...,...,...,...,...,...,...
1229173,12619596,2021-06-12T12:16:13,0x8c1f6c5639b304eb0087c2b6d5b65e32ee4f1a51e8b0...,112,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,92432639474,1157920892373161954235709850086879078532699846...,1615736234513681125600943820478722,25998097729965548918,198469
1229192,12619598,2021-06-12T12:16:25,0xac8e34aa704a1ab4068bd93d4b5aef0fb7655d292fc9...,3,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,76760509615,1157920892373161954235709850086879078532699846...,1615639241428978886102798535892145,25997255949450154096,198468
1229194,12619598,2021-06-12T12:16:25,0x0752c6529f939f046c7f9648e8d27d76b1cf767d6902...,8,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,482579746068,1157920892373161954235709850086879078532699846...,1615029729696907486273078115160620,25997255949450154096,198460
1229198,12619598,2021-06-12T12:16:25,0xbd9193ff70e4828a43679f955c9707fa63fe1dabeed3...,45,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,679981136491,1157920892373161954235709850086879078532699846...,1614171674604720940565192048438458,25997255949450154096,198449


In Uniswap V3, we can get pool's price from pool's tick.

In [307]:
def convert_price(row):
    # USDC/WETH pool has reverse token order, so let's flip it WETH/USDC
    tick = row["tick"]
    return pool_details.convert_price_to_human(tick, reverse_token_order=True)

df = df.copy(deep=True)  # https://stackoverflow.com/a/60885847/315168
df["price"] = df.apply(convert_price, axis=1)
df

,block_number,timestamp,tx_hash,log_index,pool_contract_address,amount0,amount1,sqrt_price_x96,liquidity,tick,price
19,12371376,2021-05-05T01:56:23,0xce7c3c307d820785caa12938012372fc9366a614a6aa...,26,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,1157920892373161954235709850086879078532699846...,100000000000000,1377932816571815120446551350158799,4303369674465501,195285,0.000302
21,12373132,2021-05-05T08:23:26,0x9a1c51b0bffbf840948f3b6e3f3e495ba1cd3fa64854...,192,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,1157920892373161954235709850086879078532699846...,50000000000000000,1378850591292581266780357299649652,4303369674465501,195298,0.000303
25,12373520,2021-05-05T09:50:51,0xc58715c62a5bf70a6ca09f0e51546d6cad76c8d4fff0...,8,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,1157920892373161954235709850086879078532699846...,100000000000000,1378852426842022799073024911548633,4303369674465501,195298,0.000303
26,12374077,2021-05-05T11:59:57,0x288c21b8b4fbf449b1d086a06e43b124ac2bc088c3f5...,86,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,2,1157920892373161954235709850086879078532699846...,1378852426842016741051966412054516,4304946248093346,195298,0.000303
45,12375770,2021-05-05T18:09:23,0xf76f19b991131f0f396f116f79215637dacad3cf5e1b...,249,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,1157920892373161954235709850086879078532699846...,300000000000000,1370242002332554394456531434422627,52976694435070763,195173,0.000299
...,...,...,...,...,...,...,...,...,...,...,...
1229173,12619596,2021-06-12T12:16:13,0x8c1f6c5639b304eb0087c2b6d5b65e32ee4f1a51e8b0...,112,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,92432639474,1157920892373161954235709850086879078532699846...,1615736234513681125600943820478722,25998097729965548918,198469,0.000416
1229192,12619598,2021-06-12T12:16:25,0xac8e34aa704a1ab4068bd93d4b5aef0fb7655d292fc9...,3,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,76760509615,1157920892373161954235709850086879078532699846...,1615639241428978886102798535892145,25997255949450154096,198468,0.000416
1229194,12619598,2021-06-12T12:16:25,0x0752c6529f939f046c7f9648e8d27d76b1cf767d6902...,8,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,482579746068,1157920892373161954235709850086879078532699846...,1615029729696907486273078115160620,25997255949450154096,198460,0.000416
1229198,12619598,2021-06-12T12:16:25,0xbd9193ff70e4828a43679f955c9707fa63fe1dabeed3...,45,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,679981136491,1157920892373161954235709850086879078532699846...,1614171674604720940565192048438458,25997255949450154096,198449,0.000415


Then we can convert linear price data to [OHLC candles](https://tradingstrategy.ai/docs/glossary.html#term-OHLCV).

In [308]:
from eth_defi.research.candle import convert_to_ohlc_candles

candles = convert_to_ohlc_candles(df, time_bucket=pd.Timedelta("1D"))
candles

,open,high,low,close
timestamp,,,,
2021-05-05,0.000302,0.000303,0.000282,0.000284
2021-05-06,0.000284,0.000297,0.000278,0.000287
2021-05-07,0.000287,0.000296,0.000280,0.000288
2021-05-08,0.000287,0.000288,0.000253,0.000257
2021-05-09,0.000257,0.000266,0.000251,0.000256
2021-05-10,0.000256,0.000272,0.000239,0.000254
2021-05-11,0.000253,0.000269,0.000240,0.000240
2021-05-12,0.000240,0.000270,0.000229,0.000265
2021-05-13,0.000265,0.000289,0.000247,0.000269


In [309]:
candles.index

DatetimeIndex(['2021-05-05', '2021-05-06', '2021-05-07', '2021-05-08',
               '2021-05-09', '2021-05-10', '2021-05-11', '2021-05-12',
               '2021-05-13', '2021-05-14', '2021-05-15', '2021-05-16',
               '2021-05-17', '2021-05-18', '2021-05-19', '2021-05-20',
               '2021-05-21', '2021-05-22', '2021-05-23', '2021-05-24',
               '2021-05-25', '2021-05-26', '2021-05-27', '2021-05-28',
               '2021-05-29', '2021-05-30', '2021-05-31', '2021-06-01',
               '2021-06-02', '2021-06-03', '2021-06-04', '2021-06-05',
               '2021-06-06', '2021-06-07', '2021-06-08', '2021-06-09',
               '2021-06-10', '2021-06-11', '2021-06-12'],
              dtype='datetime64[ns]', name='timestamp', freq='D')

Now we can plot the [OHLC chart using Plotly](https://plotly.com/python/ohlc-charts/).

In [310]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Ohlc(x=candles.index,
                    open=candles['open'],
                    high=candles['high'],
                    low=candles['low'],
                    close=candles['close']))
fig.show()